In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Obesity among Adults in South East Asia 1975-2016

![](https://2rdnmg1qbg403gumla1v9i2h-wpengine.netdna-ssl.com/wp-content/uploads/sites/3/2018/11/obesityDiabCardioDisease-861999778-770x553-745x490.jpg)


# Data Cleansing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings 
warnings.simplefilter('ignore')

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data_raw = pd.read_csv('/kaggle/input/obesity-among-adults-by-country-19752016/data.csv')
data_raw.head()

In [ ]:
data_raw.shape

In [ ]:
data_raw.columns

In [ ]:
data_raw = data_raw.drop([0,1,2],axis=0)
data_raw.head()

In [ ]:
data_raw=data_raw.reset_index(drop=True)
data_raw.head()

In [ ]:
data_raw = data_raw.rename(columns={"Unnamed: 0":"Country"})
data_raw.head()

In [ ]:
data_raw_melt=data_raw.melt('Country',var_name="Year",value_name="Obesity(%)")
data_raw_melt.head()

> 

In [ ]:
data_raw_melt[['Year','Sex']]=data_raw_melt['Year'].str.split('.',expand=True)

In [ ]:
data_raw_melt.head()

In [ ]:
data_raw_melt = data_raw_melt.sort_values(by=['Country','Year'])

In [ ]:
data_raw_melt['Sex']=data_raw_melt['Sex'].map({None:'Both sexes','1':'Male','2':'Female'})

In [ ]:
data_raw_melt.head()

In [ ]:
data_splitsex = data_raw_melt[data_raw_melt['Sex']!='Both sexes']

In [ ]:
data_splitsex.head()

### The obesity percentage number extraction

In [ ]:
data_splitsex['Avg Obesity(%)'] = data_splitsex['Obesity(%)'].str.extract(pat = '(\d*.\d*)',expand=True)

In [ ]:
data_splitsex.head()

### Changing data type

The data type for average obesity percentage is still an object. So I have to change the average obesity's data type into numeric in order to do further analysis.

In [ ]:
data_splitsex.info()

In [ ]:
data_splitsex.drop(data_splitsex[data_splitsex['Obesity(%)']=='No data'].index, inplace=True)

In [ ]:
data_splitsex['Avg Obesity(%)']=pd.to_numeric(data_splitsex['Avg Obesity(%)'],downcast='float')

In [ ]:
data_splitsex.head()

The average obesity percentage's data type is a float now

In [ ]:
data_splitsex.info()

In [ ]:
data_splitsex.drop('Obesity(%)',axis=1,inplace=True)

In [ ]:
data_splitsex.head()

# Obesity among male all over the world 1975-2016

Changing the dataframe format to make a better time series data visualization

In [ ]:
df_male=data_splitsex[data_splitsex['Sex']=='Male']

In [ ]:
df_male=df_male.pivot(index='Year', columns='Country', values='Avg Obesity(%)')

In [ ]:
df_male.head()

# Obesity among female all over countries 1975-2016

In [ ]:
df_female=data_splitsex[data_splitsex['Sex']=='Female']

In [ ]:
df_female=df_female.pivot(index='Year',columns='Country',values='Avg Obesity(%)')

In [ ]:
df_female.head()

# Obesity Trend Among Adults in South East Asia 1975-2016

Extracting south east asian countries data only

### Male

In [ ]:
df_SEA_Male=df_male.loc[:,['Indonesia','Singapore','Philippines','Myanmar','Thailand','Viet Nam','Malaysia','Cambodia','Brunei Darussalam']]

In [ ]:
df_SEA_Male.head()

In [ ]:
df_SEA_Male.plot(kind ='line',figsize=(18,5))
plt.title('Male Obesity Percentage in South East Asia 1975-2016')
plt.ylabel('% Obesity')
plt.xlabel('Years')
plt.show()

as seen from line chart above, Brunei Darussalam and Malaysia have the most male obesity case among south east asian countries. Starting from 1995 the trend is increasing significantly . on the other hand, Vietnam as the country with the least male obesity case, is relative stable. There is no significant change from 1975 untill 2016.

## Female

In [ ]:
df_SEA_Female=df_female.loc[:,['Indonesia','Singapore','Philippines','Myanmar','Thailand','Viet Nam','Malaysia','Cambodia','Brunei Darussalam']]

In [ ]:
df_SEA_Female.head()

In [ ]:
df_SEA_Female.plot(kind ='line',figsize=(18,5))
plt.title('Female Obesity Percentage in SEA 1975-2016')
plt.ylabel('% Obesity')
plt.xlabel('Years')
plt.show()

as seen from line chart above, Brunei Darussalam and Malaysia have the most female obesity case among south east asian countries. Starting from 1995 the trend is increasing significantly . On the other hand, Vietnam as the country with the least female obesity case, is relative stable. There is no significant change from 1975 untill 2016.

As seen from two line charts above,compared to male, there are a lot more obesity case among female in south east asian countries. 

## Obesity Male vs Female in Indonesia 1975-2016

In [ ]:
df_male_idn=df_male.loc[:,['Indonesia']]

In [ ]:
df_male_idn=df_male_idn.rename(columns={'Indonesia':'Idn Male'})

In [ ]:
df_female_idn=df_female.loc[:,['Indonesia']]

In [ ]:
df_female_idn=df_female_idn.rename(columns={'Indonesia':'Idn Female'})

In [ ]:
df_Idn=pd.concat([df_male_idn,df_female_idn],axis=1)

In [ ]:
df_Idn.head()

In [ ]:
df_Idn.plot(kind ='line')
plt.title('Male vs Female Obesity Percentage in Indonesia 1975-2016')
plt.ylabel('% Obesity')
plt.xlabel('Years')
plt.show()

# Obesity Choropleth Map for South East Asia 2016

In [ ]:
import folium

In [ ]:
url = 'https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/'
sea_geo = f'{url}/public/data/southeast-asia.geojson'

In [ ]:
sea_filter=['Indonesia','Singapore','Philippines','Myanmar','Thailand','Viet Nam','Malaysia','Cambodia','Brunei Darussalam']
data_SEA=data_splitsex[data_splitsex.Country.isin(sea_filter)]

In [ ]:
data_SEA_male2016=data_SEA[(data_SEA['Year']=='2016')&(data_SEA['Sex']=='Male')]

In [ ]:
data_SEA_male2016

In [ ]:
data_SEA_male2016 = data_SEA_male2016.replace({'Viet Nam':'Vietnam'})

In [ ]:
data_SEA_male2016

In [ ]:
m = folium.Map(location = [-2.21797,115.66283], zoom_start = 3.5)

folium.Choropleth(
    geo_data = sea_geo,
    data = data_SEA_male2016,
    columns = ['Country','Avg Obesity(%)'],
    key_on ='feature.properties.name',
    fill_color ='YlOrRd',
    fill_opacity =0.7,
    line_opacity =0.2,
    legend_name = 'Obesity Cases South East Asia 2016'
).add_to(m)
folium.LayerControl().add_to(m)

In [ ]:
m